# Context Managers
- When opening files or acquiring locks, **resources** must be **released** even if errors occur.  
- Manual `try...finally` ensures cleanup but adds **boilerplate** and potential for mistakes.  
- Forgetting to initialize the resource variable or to call cleanup in every exit path leads to **leaks**, **deadlocks**, or **corrupted data**.  
- Cleaner patterns reduce noise and risk in automation scripts.  

In [3]:
# Manual `try-catch` approach
from typing import TextIO


f: TextIO | None = None

try:
    f = open("my_log.txt", "w")
    f.write("First line\n")
    result = 1 / 0              # simulate an error
    f.write("Second line\n")
except Exception as e:
    print(f"Error: {e}")
finally:
    if f:
        print("Closing file.")
        f.close()

if f:
    print(f"File closed: {f.closed}")

Error: division by zero
Closing file.
File closed: True


## The `with` Statement Simplifies Cleanup
- The `with` statement handles **setup** and **teardown** automatically for context managers.  
- For file I/O, `with open(...) as f:` guarantees `f.close()` on block exit, even if an exception is raised.  
- Syntax is concise and idiomatic, reducing boilerplate and improving readability.  

## Common Context Manager Use cases
- **Files:** `with open(...) as f:` for automatic file closing.  
- **Locks:** `with threading.Lock():` acquires and releases locks safely.  
- **Tempfiles/Dirs:** `with tempfile.TemporaryDirectory() as d:` creates and cleans up temporary directories.  
- Context managers from the standard library cover most resource-management needs.  

In [6]:
f: TextIO | None = None

try:
    with open("my_log.txt", "w") as f:
        f.write("First line\n")
        result = 1 / 0               # simulate an error
        f.write("Second line\n")
except Exception as e:
    print(f"Error: {e}")

if f:
    print(f"File closed: {f.closed}")

Error: division by zero
File closed: True


In [1]:
import tempfile, os

dir_name = None

with tempfile.TemporaryDirectory() as tempdir:
    print(f"Created temp dir: {tempdir}")
    
    dir_name = tempdir
    test_file = os.path.join(tempdir, "test.txt")

    with open(test_file, "w") as file:
        file.write("Hello from temp directory.")

    print(f"Files inside temp dir: {os.listdir(tempdir)}")


print(f"Dir path: {dir_name}")       # just prints the dir path (str), not the actual dir
try:
    contents = os.listdir(dir_name)  # actual dir no longer exist outside the 'with' block
    print(f"Contents of {dir_name}: {contents}")
except FileNotFoundError as e:
    print(f"Expected error accessing removed directory: {e}")

Created temp dir: /var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/tmp1xfqptoq
Files inside temp dir: ['test.txt']
Dir path: /var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/tmp1xfqptoq
Expected error accessing removed directory: [Errno 2] No such file or directory: '/var/folders/zj/9tldhkbd6fd8zdsrfj84167c0000gn/T/tmp1xfqptoq'
